In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timezone, timedelta
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

In [11]:
HIST_CSV = "/content/historical_data.csv"
FG_CSV   = "/content/fear_greed_index.csv"

In [12]:
hist = pd.read_csv(HIST_CSV)
fg   = pd.read_csv(FG_CSV)

In [13]:
hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211224 entries, 0 to 211223
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Account           211224 non-null  object 
 1   Coin              211224 non-null  object 
 2   Execution Price   211224 non-null  float64
 3   Size Tokens       211224 non-null  float64
 4   Size USD          211224 non-null  float64
 5   Side              211224 non-null  object 
 6   Timestamp IST     211224 non-null  object 
 7   Start Position    211224 non-null  float64
 8   Direction         211224 non-null  object 
 9   Closed PnL        211224 non-null  float64
 10  Transaction Hash  211224 non-null  object 
 11  Order ID          211224 non-null  int64  
 12  Crossed           211224 non-null  bool   
 13  Fee               211224 non-null  float64
 14  Trade ID          211224 non-null  float64
 15  Timestamp         211224 non-null  float64
dtypes: bool(1), float64(

In [14]:
fg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2644 entries, 0 to 2643
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   timestamp       2644 non-null   int64 
 1   value           2644 non-null   int64 
 2   classification  2644 non-null   object
 3   date            2644 non-null   object
dtypes: int64(2), object(2)
memory usage: 82.8+ KB


In [15]:
sns.set(style="whitegrid")
OUTPUT_DIR = "./outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [16]:
if 'hist' not in globals():
    hist = pd.read_csv(HIST_CSV, dtype=str)   # read as str first to clean safely
else:
    hist = hist.copy()

if 'fg' not in globals():
    fg = pd.read_csv(FG_CSV, dtype=str)
else:
    fg = fg.copy()

In [17]:
print("hist shape:", hist.shape)
print("fg shape:", fg.shape)
print(hist.columns.tolist())
print(fg.columns.tolist())

hist shape: (211224, 16)
fg shape: (2644, 4)
['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']
['timestamp', 'value', 'classification', 'date']


In [18]:
hist = hist.rename(columns={
    'Account':'Account','Coin':'Coin','Execution Price':'Execution Price',
    'Size Tokens':'Size Tokens','Size USD':'Size USD','Side':'Side',
    'Timestamp IST':'Timestamp IST','Start Position':'Start Position',
    'Direction':'Direction','Closed PnL':'Closed PnL',
    'Transaction Hash':'Transaction Hash','Order ID':'Order ID','Crossed':'Crossed',
    'Fee':'Fee','Trade ID':'Trade ID','Timestamp':'Timestamp'
})


In [21]:
num_cols = ['Execution Price','Size Tokens','Size USD','Closed PnL','Fee','Start Position','Trade ID','Order ID']
for c in num_cols:
    if c in hist.columns:
        # Check if the column is of object (string) type before applying .str accessor
        if hist[c].dtype == 'object':
            hist[c] = pd.to_numeric(hist[c].str.replace(',','').replace('', np.nan), errors='coerce')
        else:
            # If it's already numeric, directly convert to numeric (e.g., int to float if needed, or handle existing NaNs)
            hist[c] = pd.to_numeric(hist[c], errors='coerce')

In [22]:
if 'Timestamp IST' in hist.columns:
    hist['Timestamp_IST_parsed'] = pd.to_datetime(hist['Timestamp IST'], errors='coerce')
    # Localize to Asia/Kolkata (IST) then convert to UTC
    hist['Timestamp_IST_parsed'] = hist['Timestamp_IST_parsed'].dt.tz_localize('Asia/Kolkata', ambiguous='NaT', nonexistent='NaT')
    hist['trade_datetime_utc'] = hist['Timestamp_IST_parsed'].dt.tz_convert('UTC')
    hist['trade_date_ist'] = hist['Timestamp_IST_parsed'].dt.date
else:
    hist['trade_datetime_utc'] = pd.NaT
    hist['trade_date_ist'] = pd.NaT

In [23]:
if 'Timestamp' in hist.columns:
    # convert to numeric safely
    hist['Timestamp_num'] = pd.to_numeric(hist['Timestamp'], errors='coerce')
    # detect scale: if > 1e12 -> milliseconds; >1e9 -> seconds
    sample = hist['Timestamp_num'].dropna().iloc[:100] if hist['Timestamp_num'].dropna().shape[0] > 0 else pd.Series([], dtype=float)
    if sample.shape[0]>0:
        s = sample.iloc[0]
        if s > 1e12:
            unit='ms'
        elif s > 1e9:
            unit='s'
        else:
            unit='s'
    else:
        unit='s'
    if unit=='ms':
        hist['trade_datetime_from_ts'] = pd.to_datetime(hist['Timestamp_num'], unit='ms', errors='coerce', utc=True)
    else:
        hist['trade_datetime_from_ts'] = pd.to_datetime(hist['Timestamp_num'], unit='s', errors='coerce', utc=True)
    # If trade_datetime_utc is missing, fill from this
    hist['trade_datetime_utc'] = hist['trade_datetime_utc'].fillna(hist['trade_datetime_from_ts'])
    hist['trade_date_utc'] = hist['trade_datetime_utc'].dt.date
else:
    hist['trade_datetime_from_ts'] = pd.NaT
    hist['trade_date_utc'] = pd.NaT

In [24]:
hist['Closed PnL'] = pd.to_numeric(hist['Closed PnL'], errors='coerce')
hist['is_win'] = hist['Closed PnL'] > 0
hist['pnl_sign'] = hist['Closed PnL'].apply(lambda x: 'win' if x>0 else ('loss' if (pd.notnull(x) and x<0) else 'zero'))
hist['Size USD'] = pd.to_numeric(hist['Size USD'], errors='coerce')
hist = hist.dropna(subset=['trade_datetime_utc','Account'])

In [25]:
fg = fg.rename(columns={c:c.strip() for c in fg.columns})

In [26]:
if 'date' in fg.columns:
    fg['date'] = pd.to_datetime(fg['date'], errors='coerce').dt.date
else:
    # fallback: try parse timestamp column if int epoch
    if 'timestamp' in fg.columns:
        try:
            ts_sample = int(fg['timestamp'].iloc[0])
            # heuristic: ts in seconds vs ms
            if ts_sample > 1e12:
                fg['date'] = pd.to_datetime(pd.to_numeric(fg['timestamp']), unit='ms', errors='coerce', utc=True).dt.date
            else:
                fg['date'] = pd.to_datetime(pd.to_numeric(fg['timestamp']), unit='s', errors='coerce', utc=True).dt.date
        except:
            fg['date'] = pd.NaT

In [27]:
fg['classification'] = fg['classification'].str.strip().str.title()
fg['value'] = pd.to_numeric(fg['value'], errors='coerce')


In [28]:
hist['account'] = hist['Account']
hist['date_ist'] = pd.to_datetime(hist['trade_date_ist'])
hist['join_date'] = hist['date_ist'].dt.date

daily_account = hist.groupby(['join_date','account']).agg(
    trade_count=('Trade ID','nunique'),
    total_pnl=('Closed PnL','sum'),
    mean_pnl=('Closed PnL','mean'),
    win_rate=('is_win','mean'),
    total_volume_usd=('Size USD','sum'),
    avg_trade_size_usd=('Size USD','mean'),
    avg_fee=('Fee','mean'),
    avg_execution_price=('Execution Price','mean')
).reset_index()

# ---------- Merge with Fear-Greed by date ----------
daily_account['date'] = daily_account['join_date']
fg_small = fg[['date','classification','value']].drop_duplicates(subset=['date'])
merged = daily_account.merge(fg_small, how='left', on='date')

# Save aggregated outputs
daily_account.to_csv(os.path.join(OUTPUT_DIR,'daily_account_metrics.csv'), index=False)
merged.to_csv(os.path.join(OUTPUT_DIR,'daily_account_metrics_with_sentiment.csv'), index=False)
print("Saved daily_account_metrics.* to", OUTPUT_DIR)

Saved daily_account_metrics.* to ./outputs


In [29]:
print("\nSummary by sentiment classification (aggregated over account-days):")
summary_by_sent = merged.groupby('classification').agg(
    mean_total_pnl=('total_pnl','mean'),
    median_total_pnl=('total_pnl','median'),
    mean_win_rate=('win_rate','mean'),
    mean_trade_count=('trade_count','mean'),
    n_account_days=('date','count')
).reset_index().sort_values('n_account_days', ascending=False)
print(summary_by_sent)



Summary by sentiment classification (aggregated over account-days):
  classification  mean_total_pnl  median_total_pnl  mean_win_rate  \
3          Greed     3191.793365        147.445788       0.353119   
2           Fear    11332.647023        396.062720       0.349909   
1  Extreme Greed    10329.407818        108.938997       0.335945   
4        Neutral     2017.749783         87.752181       0.312524   
0   Extreme Fear      133.331411          0.000000       0.173498   

   mean_trade_count  n_account_days  
3         50.643979             191  
2         73.089172             157  
1         42.419643             112  
4         62.216216              37  
0         57.181818              33  


In [30]:
df_test = merged[merged['classification'].isin(['Fear','Greed'])].dropna(subset=['total_pnl','win_rate'])
print("\nN rows for test:", df_test.shape[0])


N rows for test: 348


In [31]:
fear_pnl = df_test[df_test['classification']=='Fear']['total_pnl']
greed_pnl = df_test[df_test['classification']=='Greed']['total_pnl']
# normality check (large sample -> use non-parametric too)
print("\nT-test (total_pnl):")


T-test (total_pnl):


In [32]:
try:
    tstat, pval = stats.ttest_ind(fear_pnl, greed_pnl, equal_var=False, nan_policy='omit')
    print("ttest_ind: t=%.4f p=%.4e" % (tstat, pval))
except Exception as e:
    print("ttest error:", e)
print("Mann-Whitney (total_pnl):")

ttest_ind: t=1.6935 p=9.1874e-02
Mann-Whitney (total_pnl):


In [33]:
try:
    u, pu = stats.mannwhitneyu(fear_pnl, greed_pnl, alternative='two-sided')
    print("U=%.3f p=%.4e" % (u, pu))
except Exception as e:
    print("MW error:", e)

U=16134.000 p=2.1649e-01


In [34]:
# win_rate test
fear_wr = df_test[df_test['classification']=='Fear']['win_rate']
greed_wr = df_test[df_test['classification']=='Greed']['win_rate']
print("\nT-test (win_rate):")
try:
    tstat2, pval2 = stats.ttest_ind(fear_wr, greed_wr, equal_var=False, nan_policy='omit')
    print("ttest_ind: t=%.4f p=%.4e" % (tstat2, pval2))
except Exception as e:
    print("ttest error:", e)
print("Mann-Whitney (win_rate):")


T-test (win_rate):
ttest_ind: t=-0.0902 p=9.2815e-01
Mann-Whitney (win_rate):


In [35]:
try:
    u2, p2 = stats.mannwhitneyu(fear_wr, greed_wr, alternative='two-sided')
    print("U=%.3f p=%.4e" % (u2, p2))
except Exception as e:
    print("MW error:", e)

U=14931.000 p=9.4616e-01


In [36]:
global_daily = merged.groupby('date').agg(
    total_pnl_global=('total_pnl','sum'),
    mean_win_rate_global=('win_rate','mean'),
    sentiment_value=('value','mean')  # value from FG
).reset_index().sort_values('date')
global_daily = global_daily.dropna(subset=['sentiment_value','total_pnl_global'])
global_daily['sentiment_value'] = pd.to_numeric(global_daily['sentiment_value'], errors='coerce')

In [37]:
max_lag = 7
cc = {}
for lag in range(0, max_lag+1):
    shifted = global_daily['sentiment_value'].shift(lag)
    corr = global_daily['total_pnl_global'].corr(shifted)
    cc[lag] = corr
print("\nLagged correlation (total_pnl_global vs sentiment shifted by lag days):", cc)


Lagged correlation (total_pnl_global vs sentiment shifted by lag days): {0: np.float64(-0.05725161652917338), 1: np.float64(-0.12590779028038546), 2: np.float64(-0.08022913136136775), 3: np.float64(0.01570271878121309), 4: np.float64(-0.028436455255948335), 5: np.float64(5.193925445113622e-05), 6: np.float64(0.0992943456922018), 7: np.float64(0.10515211725095995)}


In [39]:
# Save lag plot
lags = list(cc.keys())
corrs = [cc[k] for k in lags]
plt.figure(figsize=(8,4))
plt.stem(lags, corrs)
plt.xlabel('Lag (days) -- sentiment shifted forward (sentiment_t -> pnl_t+lag)')
plt.ylabel('Pearson corr with total_pnl_global')
plt.title('Lagged correlation sentiment -> pnl (global)')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'lagged_correlation_sentiment_pnl.png'))
plt.close()


In [40]:
acct_agg = hist.groupby('account').agg(
    tot_trades=('Trade ID','nunique'),
    mean_pnl=('Closed PnL','mean'),
    median_pnl=('Closed PnL','median'),
    win_rate=('is_win','mean'),
    avg_size_usd=('Size USD','mean'),
    pnl_std=('Closed PnL','std')
).reset_index().fillna(0)

In [41]:
# choose k via elbow optionally; here k=4
features = ['tot_trades','mean_pnl','win_rate','avg_size_usd','pnl_std']
X = acct_agg[features].replace([np.inf,-np.inf], np.nan).fillna(0)
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
k = 4
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10).fit(Xs)
acct_agg['cluster'] = kmeans.labels_


In [42]:
# profile clusters
cluster_profile = acct_agg.groupby('cluster')[features].median().reset_index()
print("\nCluster profile (median of features):")
print(cluster_profile)
acct_agg.to_csv(os.path.join(OUTPUT_DIR,'account_clusters.csv'), index=False)


Cluster profile (median of features):
   cluster  tot_trades    mean_pnl  win_rate  avg_size_usd      pnl_std
0        0      1431.0   11.041249  0.341460   2707.579181   294.939745
1        1       561.5   81.795104  0.465559   3059.168279   471.902909
2        2       782.0  336.735548  0.345912   5189.367128  2228.336140
3        3      1411.0   40.740489  0.401193  22504.555829   639.402444


In [43]:
merged['net_profitable'] = merged['total_pnl'] > 0
# create features
# use sentiment value and classification plus trade_count, avg trade size etc.
model_df = merged[['date','account','trade_count','total_pnl','win_rate','total_volume_usd','avg_trade_size_usd','classification','value','net_profitable']].dropna()
model_df['classification_cat'] = model_df['classification'].fillna('Unknown').astype('category').cat.codes
X = model_df[['trade_count','win_rate','total_volume_usd','avg_trade_size_usd','classification_cat','value']]
y = model_df['net_profitable'].astype(int)

In [44]:
# train/test split by accounts to avoid leakage
train_idx, test_idx = train_test_split(model_df.index.unique(), test_size=0.2, random_state=42)
X_train = X.loc[train_idx]
y_train = y.loc[train_idx]
X_test = X.loc[test_idx]
y_test = y.loc[test_idx]


In [45]:
rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
pred_proba = rf.predict_proba(X_test)[:,1]

In [46]:

print("\nClassification report (Random Forest) predicting net_profitable account-day:")
print(classification_report(y_test, pred))
print("ROC AUC:", roc_auc_score(y_test, pred_proba))


Classification report (Random Forest) predicting net_profitable account-day:
              precision    recall  f1-score   support

           0       0.97      0.87      0.92        45
           1       0.91      0.98      0.94        61

    accuracy                           0.93       106
   macro avg       0.94      0.93      0.93       106
weighted avg       0.94      0.93      0.93       106

ROC AUC: 0.9451730418943535


In [47]:
# feature importance
fi = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nFeature importances:")
print(fi)
fi.to_csv(os.path.join(OUTPUT_DIR,'rf_feature_importances.csv'))



Feature importances:
win_rate              0.731230
trade_count           0.104054
total_volume_usd      0.067137
avg_trade_size_usd    0.054982
value                 0.031744
classification_cat    0.010854
dtype: float64


In [48]:
# Save a few plots
plt.figure(figsize=(8,4))
sns.boxplot(data=merged, x='classification', y='total_pnl')
plt.title('Total PnL per account-day by sentiment classification')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'pnl_by_sentiment_box.png'))
plt.close()

In [49]:

plt.figure(figsize=(8,4))
sns.barplot(x=fi.index, y=fi.values)
plt.title('RF feature importances')
plt.xticks(rotation=30)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'rf_feature_importances.png'))
plt.close()

print("All outputs and plots saved to", OUTPUT_DIR)

All outputs and plots saved to ./outputs
